# Numerical Errors in Energy Tensor Calculations

This notebook demonstrates various types of numerical errors that arise when computing stress-energy tensors from spacetime metrics. Understanding these errors is crucial for accurate analysis of warp drive metrics and other exotic spacetimes.

## Types of Errors

1. **Edge of grid errors** - Boundary effects where finite differences cannot be fully evaluated
2. **Finite difference discretization error** - Resolution-dependent errors from grid spacing
3. **Floating point round-off error** - Machine precision limitations
4. **Finite difference truncation error** - Approximation errors from finite difference order

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from warpfactory.metrics.schwarzschild import get_schwarzschild_metric
from warpfactory.solver.energy import get_energy_tensor

## Schwarzschild Example

The Schwarzschild metric provides an excellent test case for numerical error analysis because:
- It has a known analytical solution
- Outside the Schwarzschild radius, the stress-energy tensor should be exactly zero (vacuum solution)
- Any non-zero energy density computed is purely due to numerical errors

We will create three different versions of the Schwarzschild metric to compare:
1. **Low resolution** (20x5x5) with fourth-order finite differences
2. **High resolution** (2000x5x5) with fourth-order finite differences  
3. **High resolution** (2000x5x5) with second-order finite differences

In [ ]:
# Make low resolution Schwarzschild metric
grid_size_low = [1, 20, 5, 5]
grid_scaling_low = [1, 1, 1, 1]
world_center_low = [(grid_size_low[i] + 1) / 2 * grid_scaling_low[i] for i in range(4)]
rs_low = 0.0001

low_res_schwarzschild = get_schwarzschild_metric(
    grid_size=grid_size_low,
    world_center=world_center_low,
    schwarzschild_radius=rs_low,
    grid_scaling=grid_scaling_low
)

print(f"Low resolution metric: {grid_size_low}")
print(f"  Grid scaling: {grid_scaling_low}")
print(f"  World center: {world_center_low}")
print(f"  Schwarzschild radius: {rs_low}")

In [ ]:
# Make high resolution Schwarzschild metric
grid_size_high = [1, 2000, 5, 5]
grid_scaling_high = [1, 0.01, 0.01, 0.01]
world_center_high = [(grid_size_high[i] + 1) / 2 * grid_scaling_high[i] for i in range(4)]
rs_high = 0.0001

high_res_schwarzschild = get_schwarzschild_metric(
    grid_size=grid_size_high,
    world_center=world_center_high,
    schwarzschild_radius=rs_high,
    grid_scaling=grid_scaling_high
)

print(f"High resolution metric: {grid_size_high}")
print(f"  Grid scaling: {grid_scaling_high}")
print(f"  World center: {world_center_high}")
print(f"  Schwarzschild radius: {rs_high}")

In [ ]:
# Make second-order high resolution Schwarzschild metric
grid_size_second = [1, 2000, 5, 5]
grid_scaling_second = [1, 0.01, 0.01, 0.01]
world_center_second = [(grid_size_second[i] + 1) / 2 * grid_scaling_second[i] for i in range(4)]
rs_second = 0.0001

second_order_schwarzschild = get_schwarzschild_metric(
    grid_size=grid_size_second,
    world_center=world_center_second,
    schwarzschild_radius=rs_second,
    grid_scaling=grid_scaling_second
)

print(f"Second-order high resolution metric: {grid_size_second}")
print(f"  Grid scaling: {grid_scaling_second}")
print(f"  World center: {world_center_second}")
print(f"  Schwarzschild radius: {rs_second}")

## Compute Energy Tensors

Now we compute the stress-energy tensors for all three metrics. We use different finite difference orders to demonstrate truncation error effects.

In [ ]:
# Compute energy tensors with different accuracies
print("Computing energy tensors...")

# Low resolution with fourth-order
low_res_energy = get_energy_tensor(low_res_schwarzschild, order='fourth')
print("  Low resolution (fourth-order) complete")

# High resolution with fourth-order
high_res_energy = get_energy_tensor(high_res_schwarzschild, order='fourth')
print("  High resolution (fourth-order) complete")

# High resolution with second-order
second_order_energy = get_energy_tensor(second_order_schwarzschild, order='second')
print("  High resolution (second-order) complete")

print("\nAll energy tensor calculations complete!")

## Plot Energy Tensors

We plot the energy density $|T^{00}|$ along the x-axis for all three cases on a logarithmic scale. This visualization reveals all four types of numerical errors simultaneously.

In [ ]:
# Extract energy density along x-axis at center of y and z
def get_energy_density_profile(energy_tensor, grid_size, grid_scaling, world_center):
    """Extract energy density T^00 along x-axis at spatial center."""
    t_idx = 0
    y_idx = int(np.round(world_center[2] / grid_scaling[2]))
    z_idx = int(np.round(world_center[3] / grid_scaling[3]))
    
    # Extract energy density component
    energy_density = energy_tensor.tensor[(0, 0)][t_idx, :, y_idx, z_idx]
    
    # Create x-coordinates
    x_coords = np.arange(1, grid_size[1] + 1) * grid_scaling[1]
    
    return x_coords, np.abs(energy_density)

# Get profiles for all three cases
x_low, energy_low = get_energy_density_profile(
    low_res_energy, grid_size_low, grid_scaling_low, world_center_low
)
x_high, energy_high = get_energy_density_profile(
    high_res_energy, grid_size_high, grid_scaling_high, world_center_high
)
x_second, energy_second = get_energy_density_profile(
    second_order_energy, grid_size_second, grid_scaling_second, world_center_second
)

In [ ]:
# Create comparison plot
plt.figure(figsize=(14, 8))

plt.semilogy(x_low, energy_low, 'o-', label='Low Res Fourth-Order', linewidth=2, markersize=5)
plt.semilogy(x_high, energy_high, '-', label='High Res Fourth-Order', linewidth=1.5, alpha=0.8)
plt.semilogy(x_second, energy_second, '-', label='High Res Second-Order', linewidth=1.5, alpha=0.8)

plt.xlim([-1, 21])
plt.xlabel('x coordinate', fontsize=12)
plt.ylabel(r'$|T^{00}|$ (Energy Density)', fontsize=12)
plt.title('Numerical Errors in Schwarzschild Energy Tensor Calculation', fontsize=14)
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("\nAll four types of errors are visible in this single plot!")

## Error Analysis

The plot above reveals all four types of numerical errors simultaneously. Since the Schwarzschild metric should produce zero energy density outside the Schwarzschild radius, any computed non-zero values represent numerical errors.

## 1. Edge of Grid Error

**Observation:** Large spikes at both ends of the plot (x ≈ 0 and x ≈ 20)

**Cause:** At the boundaries of the computational domain, finite difference stencils extend beyond the grid. The code cannot evaluate complete derivatives at these locations, leading to significant errors.

**Solution:** Exclude a boundary region of at least 2-3 grid points when analyzing energy tensors and energy conditions. These boundary regions should not be used for physical interpretation.

**Impact:** High magnitude errors confined to grid edges

In [ ]:
# Demonstrate edge effects quantitatively
edge_width = 3  # Number of points at edge

print("Edge of Grid Error Analysis:")
print("="*60)

for name, x, energy in [("Low Res Fourth", x_low, energy_low),
                         ("High Res Fourth", x_high, energy_high),
                         ("High Res Second", x_second, energy_second)]:
    
    # Edge region errors
    left_edge = energy[:edge_width]
    right_edge = energy[-edge_width:]
    
    # Interior region errors (excluding edges)
    interior = energy[edge_width:-edge_width]
    
    print(f"\n{name}:")
    print(f"  Left edge max error:  {np.max(left_edge):.3e}")
    print(f"  Right edge max error: {np.max(right_edge):.3e}")
    print(f"  Interior max error:   {np.max(interior):.3e}")
    print(f"  Edge/Interior ratio:  {np.max([np.max(left_edge), np.max(right_edge)]) / np.max(interior):.1f}x")

## 2. Finite Difference Discretization Error

**Observation:** The low-resolution case (blue line) shows higher errors over a wider spatial region compared to the high-resolution cases

**Cause:** When the rate of change of metric components approaches or exceeds the grid resolution, the finite difference approximation breaks down. This is particularly visible near the Schwarzschild radius where the metric changes rapidly.

**Solution:** 
- Use finer grid spacing in regions of rapid metric variation
- Implement adaptive mesh refinement
- Ensure grid resolution is sufficient to resolve all relevant length scales

**Impact:** Resolution-dependent errors that decrease with grid refinement

In [ ]:
# Convergence study: Compare errors between resolutions
print("Finite Difference Discretization Error Analysis:")
print("="*60)

# Compute error in interior region only (excluding edges)
edge_exclude = 5

energy_low_interior = energy_low[edge_exclude:-edge_exclude]
energy_high_interior = energy_high[edge_exclude:-edge_exclude]

print(f"\nResolution comparison:")
print(f"  Low resolution grid spacing:  {grid_scaling_low[1]:.3f}")
print(f"  High resolution grid spacing: {grid_scaling_high[1]:.3f}")
print(f"  Refinement factor: {grid_scaling_low[1] / grid_scaling_high[1]:.0f}x")

print(f"\nInterior error levels:")
print(f"  Low resolution mean error:  {np.mean(energy_low_interior):.3e}")
print(f"  High resolution mean error: {np.mean(energy_high_interior):.3e}")
print(f"  Error reduction factor:     {np.mean(energy_low_interior) / np.mean(energy_high_interior):.1f}x")

# Theoretical convergence rate for 4th order method should be ~h^4
theoretical_reduction = (grid_scaling_low[1] / grid_scaling_high[1])**4
print(f"  Theoretical reduction (4th order): {theoretical_reduction:.1f}x")

In [ ]:
# Visualize discretization error with zoom on transition region
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.semilogy(x_low, energy_low, 'o-', label='Low Res (Δx=1.0)', linewidth=2, markersize=6)
plt.semilogy(x_high, energy_high, '-', label='High Res (Δx=0.01)', linewidth=1.5)
plt.xlim([5, 15])
plt.ylim([1e-35, 1e-25])
plt.xlabel('x coordinate', fontsize=11)
plt.ylabel(r'$|T^{00}|$', fontsize=11)
plt.title('Discretization Error (Zoomed)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, which='both')

plt.subplot(1, 2, 2)
plt.plot(x_low, energy_low / np.mean(energy_high_interior), 'o-', linewidth=2, markersize=6)
plt.xlabel('x coordinate', fontsize=11)
plt.ylabel('Error Ratio (Low/High)', fontsize=11)
plt.title('Relative Error Due to Discretization', fontsize=12)
plt.xlim([5, 15])
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Floating Point Round-Off Error

**Observation:** Errors become very noisy around 10⁻³² in the plot

**Cause:** IEEE 754 double precision floating point numbers have approximately 16 decimal digits of precision. When values span many orders of magnitude, the least significant digits are lost. The maximum dynamic range is limited by the largest values stored in the array.

**Solution:**
- Use extended precision (e.g., 128-bit floats) for critical calculations
- Normalize intermediate calculations to keep values within reasonable ranges
- Be aware of the precision floor when interpreting small values

**Impact:** Irreducible noise floor around machine epsilon × max(array)

In [ ]:
# Analyze floating point precision limits
print("Floating Point Round-Off Error Analysis:")
print("="*60)

# Machine epsilon for double precision
machine_eps = np.finfo(np.float64).eps
print(f"\nMachine epsilon (double precision): {machine_eps:.3e}")
print(f"This represents ~{-np.log10(machine_eps):.1f} decimal digits of precision")

# Find the noise floor in each dataset
for name, energy in [("Low Res Fourth", energy_low),
                     ("High Res Fourth", energy_high),
                     ("High Res Second", energy_second)]:
    
    max_val = np.max(np.abs(energy))
    min_val = np.min(energy[energy > 0])  # Exclude exact zeros
    
    dynamic_range = max_val / min_val
    precision_floor = max_val * machine_eps
    
    print(f"\n{name}:")
    print(f"  Maximum value:     {max_val:.3e}")
    print(f"  Minimum value:     {min_val:.3e}")
    print(f"  Dynamic range:     {dynamic_range:.3e} ({np.log10(dynamic_range):.1f} orders of magnitude)")
    print(f"  Precision floor:   {precision_floor:.3e}")
    print(f"  Values below floor: {np.sum(energy < precision_floor)} / {len(energy)}")

In [ ]:
# Visualize the precision floor
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.semilogy(x_high, energy_high, '-', label='High Res Fourth-Order', linewidth=1.5)
plt.axhline(y=np.max(energy_high) * machine_eps, color='r', linestyle='--', 
            label=f'Precision Floor ({machine_eps:.1e} × max)', linewidth=2)
plt.xlim([5, 15])
plt.ylim([1e-35, 1e-25])
plt.xlabel('x coordinate', fontsize=11)
plt.ylabel(r'$|T^{00}|$', fontsize=11)
plt.title('Floating Point Precision Floor', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, which='both')

plt.subplot(1, 2, 2)
# Show noise characteristics in the low-error region
interior_idx = slice(500, 1500)
relative_noise = energy_high[interior_idx] / np.mean(energy_high[interior_idx])
plt.hist(np.log10(relative_noise), bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('log₁₀(Relative Error)', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title('Relative Error Distribution (Interior Region)', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Finite Difference Truncation Error

**Observation:** The yellow (second-order) line shows consistently higher errors than the orange (fourth-order) line throughout the interior region

**Cause:** Finite difference approximations are truncated Taylor series. Higher-order methods include more terms, providing better approximations. Second-order methods have O(h²) error, while fourth-order methods have O(h⁴) error.

**Solution:**
- Use higher-order finite difference schemes when accuracy is critical
- Balance computational cost against required accuracy
- For very smooth functions, spectral methods may be more appropriate

**Impact:** Systematic error that scales with grid spacing to a power determined by the method order

In [ ]:
# Compare truncation errors between 2nd and 4th order methods
print("Finite Difference Truncation Error Analysis:")
print("="*60)

# Both use same resolution, so difference is purely from method order
edge_exclude = 5

energy_fourth_interior = energy_high[edge_exclude:-edge_exclude]
energy_second_interior = energy_second[edge_exclude:-edge_exclude]

print(f"\nBoth methods use identical grid:")
print(f"  Grid size: {grid_size_high}")
print(f"  Grid spacing: {grid_scaling_high[1]:.3f}")

print(f"\nInterior error comparison:")
print(f"  Fourth-order mean error: {np.mean(energy_fourth_interior):.3e}")
print(f"  Second-order mean error: {np.mean(energy_second_interior):.3e}")
print(f"  Error ratio (2nd/4th):   {np.mean(energy_second_interior) / np.mean(energy_fourth_interior):.1f}x")

print(f"\nPeak error comparison:")
print(f"  Fourth-order max error:  {np.max(energy_fourth_interior):.3e}")
print(f"  Second-order max error:  {np.max(energy_second_interior):.3e}")
print(f"  Peak ratio (2nd/4th):    {np.max(energy_second_interior) / np.max(energy_fourth_interior):.1f}x")

In [ ]:
# Visualize truncation error differences
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.semilogy(x_high, energy_high, '-', label='Fourth-Order O(h⁴)', linewidth=2)
plt.semilogy(x_second, energy_second, '-', label='Second-Order O(h²)', linewidth=2)
plt.xlim([5, 15])
plt.ylim([1e-35, 1e-28])
plt.xlabel('x coordinate', fontsize=11)
plt.ylabel(r'$|T^{00}|$', fontsize=11)
plt.title('Truncation Error: 2nd vs 4th Order', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, which='both')

plt.subplot(1, 2, 2)
# Show ratio of errors
# Only compare where both are above noise floor
valid_comparison = (energy_high > 1e-34) & (energy_second > 1e-34)
error_ratio = np.zeros_like(energy_high)
error_ratio[valid_comparison] = energy_second[valid_comparison] / energy_high[valid_comparison]

plt.plot(x_high[valid_comparison], error_ratio[valid_comparison], '-', linewidth=1.5)
plt.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Equal Error')
plt.xlabel('x coordinate', fontsize=11)
plt.ylabel('Error Ratio (2nd Order / 4th Order)', fontsize=11)
plt.title('Relative Truncation Error', fontsize=12)
plt.xlim([5, 15])
plt.ylim([0, 50])
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nMedian error ratio in interior: {np.median(error_ratio[valid_comparison]):.1f}x")

## Summary and Best Practices

This analysis demonstrates four fundamental types of numerical errors in energy tensor calculations:

### Error Summary Table

| Error Type | Cause | Mitigation | Observable Feature |
|------------|-------|------------|--------------------|
| **Edge Effects** | Incomplete stencils at boundaries | Exclude 2-3 boundary layers | Large spikes at grid edges |
| **Discretization** | Insufficient grid resolution | Refine mesh in high-gradient regions | Resolution-dependent errors |
| **Round-off** | Floating point precision limits | Use extended precision if needed | Noise floor ~10⁻³² |
| **Truncation** | Finite difference approximation | Use higher-order schemes | Systematic error scaling with h |

### Recommendations for Energy Tensor Calculations:

1. **Always exclude boundary regions** (at least 2-3 grid points) from analysis
2. **Perform convergence studies** to verify sufficient grid resolution
3. **Use fourth-order finite differences** for improved accuracy (default in WarpFactory)
4. **Be aware of the precision floor** when interpreting very small values
5. **Validate results** using metrics with known analytical solutions

### Physical Interpretation

When analyzing exotic spacetimes:
- Energy condition violations must exceed numerical error levels to be physically meaningful
- Very small violations (< 10⁻³⁰ in geometric units) may be purely numerical artifacts
- Resolution studies help distinguish physical features from numerical artifacts
- Sharp features in metrics require particularly fine resolution to resolve accurately

In [ ]:
# Create comprehensive error summary plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comprehensive Numerical Error Analysis', fontsize=16, fontweight='bold')

# 1. Edge effects
ax = axes[0, 0]
ax.semilogy(x_low, energy_low, 'o-', label='Low Res Fourth', linewidth=2, markersize=5)
ax.axvspan(-1, 2, alpha=0.2, color='red', label='Edge Region')
ax.axvspan(18, 21, alpha=0.2, color='red')
ax.set_xlim([-1, 21])
ax.set_xlabel('x coordinate', fontsize=11)
ax.set_ylabel(r'$|T^{00}|$', fontsize=11)
ax.set_title('1. Edge of Grid Error', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')

# 2. Discretization error
ax = axes[0, 1]
ax.semilogy(x_low, energy_low, 'o-', label='Δx = 1.0', linewidth=2, markersize=5)
ax.semilogy(x_high, energy_high, '-', label='Δx = 0.01', linewidth=2)
ax.set_xlim([5, 15])
ax.set_ylim([1e-35, 1e-25])
ax.set_xlabel('x coordinate', fontsize=11)
ax.set_ylabel(r'$|T^{00}|$', fontsize=11)
ax.set_title('2. Finite Difference Discretization Error', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')

# 3. Round-off error
ax = axes[1, 0]
ax.semilogy(x_high, energy_high, '-', label='High Res Data', linewidth=2)
precision_floor = np.max(energy_high) * machine_eps
ax.axhline(y=precision_floor, color='r', linestyle='--', 
           label=f'Precision Floor ≈ 10⁻³²', linewidth=2)
ax.fill_between(x_high, 1e-36, precision_floor, alpha=0.2, color='red')
ax.set_xlim([5, 15])
ax.set_ylim([1e-36, 1e-25])
ax.set_xlabel('x coordinate', fontsize=11)
ax.set_ylabel(r'$|T^{00}|$', fontsize=11)
ax.set_title('3. Floating Point Round-Off Error', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')

# 4. Truncation error
ax = axes[1, 1]
ax.semilogy(x_high, energy_high, '-', label='Fourth-Order O(h⁴)', linewidth=2)
ax.semilogy(x_second, energy_second, '-', label='Second-Order O(h²)', linewidth=2)
ax.set_xlim([5, 15])
ax.set_ylim([1e-35, 1e-28])
ax.set_xlabel('x coordinate', fontsize=11)
ax.set_ylabel(r'$|T^{00}|$', fontsize=11)
ax.set_title('4. Finite Difference Truncation Error', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("ERROR ANALYSIS COMPLETE")
print("="*70)
print("\nAll four error types have been demonstrated and quantified.")
print("Use these insights to interpret energy tensor calculations in")
print("your own warp drive and exotic spacetime research.")